In [ ]:
from os.path import basename

files = {}
with open("PSP/pq3s") as pq3s:
    for line in pq3s:
        _,_,size,name = line.split()
        files[basename(name).split(".")[0]] = { "qb3": int(size) }

with open("PSP/JP2s") as JP2s:
    for line in JP2s:
        _,_,size,name = line.split()
        files[basename(name).split(".")[0]]["jp2"] = int(size)


sizes = list((files[f]['jp2'], files[f]['qb3']) for f in files.keys())

In [ ]:
def plot_size(sizes):
    from matplotlib import pyplot as plt
    import numpy as np

    x, y = zip(*sizes)
    coef = np.polyfit(x, y, 1)
    polifn = np.poly1d(coef)

    plt.figure(figsize=(10, 10 * coef[0]))
    plt.scatter(x,y)
    plt.plot(x, polifn(x), color='red', label = f"1:{coef[0]}")
    plt.plot((0,3e9), (0,3e9), color='black', label = "1:1")
    plt.legend()
    plt.show()

In [ ]:
# RED files
plot_size([(files[f]['jp2'], files[f]['qb3']) for f in files.keys() if "RED" not in f])

In [ ]:
# Color files
plot_size([(files[f]['jp2'], files[f]['qb3']) for f in files.keys() if "RED" in f])

In [ ]:
# Plot band histrograms
def get_values(fname : str):

    try:
        import xml.etree.ElementTree as ET
        tree = ET.parse(fname)
        root = tree.getroot()
        pam=root.find("PAMDataset")
        seq = []
        for child in pam:
            if child.tag == "PAMRasterBand":
                hist = child.find("Histograms/HistItem")
                count = int(hist.find("./BucketCount").text)
                hist = [int(c) for c in hist.find(f"./HistCounts").text.split("|")]
                if len(hist) == count:
                    seq.append(hist)
                else:
                    raise Exception(f"{fname} has {len(hist)} instead of {count}")
        if (len(seq) == 0):
            raise Exception(f"{fname} has no band")
        if (len(seq) == 1):
            return seq[0]
        else:
            return seq
    except:
        return None

def plot_hist(hist: list):
    import matplotlib.pyplot as plt
    colors = ["red", "green", "blue"]
    if type(hist[0]) == int:
        hist[0] = 0
        plt.figure(figsize=(20, 10))
        plt.plot(hist, color=colors[0], label = "RED")
        plt.show()
    else:
        plt.figure(figsize = (20, 10))
        band = 1
        for seq in hist:
            seq[0]  = 0
            plt.plot(seq, label=f"Band {band} : {len(seq)}", color=colors[band-1])
            band += 1
        plt.legend()
        plt.show()


In [ ]:
plot_hist(get_values("PSP/ORB_010500_010599/PSP_010573_1755/PSP_010573_1755_RED.mrf"))

In [ ]:
plot_hist(get_values("PSP/ORB_010500_010599/PSP_010573_1755/PSP_010573_1755_COLOR.mrf"))